<div style="text-align: center">
    <img src="https://github.com/michal-owsiak/research-on-psychedelic-and-non-psychedelic-mystical-experiences/raw/main/public/banner.png" width="600" alt="Banner" style="border-radius: 8px">
    <h1><strong>A study of Poles' psychedelic and non-psychedelic mystical experiences in relation to fear of death, emotional empathy, and life satisfaction</strong></h1>
</div>
<div style="text-align: center">
    <h3><strong>Part II: Lingustic analysis</strong></h1>
</div>